Note: This doesn't work for any commits after 5/1/2023 because I reworked the GRS.py module. 

Outputs are maintained for proof of concept, but to reproduce this as is, revert to a pre 5/1/2023 commit.

---

Make sure normalization works as intended (check against manual value)

In [11]:
import sys
import os
import math
sys.path.append("..")

from lib.GRS import GRS as grs

In [5]:
GRS = grs('/home/lain/sync/01_Research/Mars_Magnetics/code/05_mag_src_map/src/data/GRS')
GRS.loadData()

In [6]:
## Optional test: verify that normalization works as intended:

lon = -90
lat = -30

k = GRS.getConcentration(lon,lat,"th", normalized=False)
print(f'raw k: {k}')
k_norm = GRS.getConcentration(lon,lat,"th", normalized=True)
print(f'normalized k: {k_norm}')
print()

h2o = GRS.getConcentration(lon,lat,"h2o", normalized=False)
cl = GRS.getConcentration(lon,lat,"cl", normalized=False)
si = GRS.getConcentration(lon,lat,"si", normalized=False)
print(f'h2o: {h2o}')
print(f'cl: {cl}')
print(f'si: {si}')
print(f'sum volatiles: {h2o+cl+si}')
print()

k_norm_man = k / (1-(h2o+cl+si))
print(f'manual normalization: {k_norm_man}')
print(f'equal?: {k_norm == k_norm_man}')

raw k: 4.5225000000000004e-07
normalized k: 5.959420595414969e-07

h2o: 0.024184999999999998
cl: 0.0038525
si: 0.21308
sum volatiles: 0.24111749999999998

manual normalization: 5.959420595414969e-07
equal?: True


---
## now check H calculation

In [8]:
henry = {
    "lon": 23.4,
    "lat": 10.8,
    "formation": 3.6e9,
    "diameter_km": 167.58,
}

In [16]:
lon = henry['lon']+1
lat = henry['lat']+1

HPE = {
    # Constants from:
    #     - "Amagmantic Hydrothermal Systems on Mars from Radiogenic Heat", by Ojha et al. 2021,
    #     - "Geodynamics", by Turcotte & Schubert 2014.
    "U238": {
        "isotopic_frac": 0.9928, # natural abundance of this isotope relative to all isotopes of this element
        "heat_release_const": 9.46e-5, # net energy per unit mass [W/kg]
        "half_life": 4.47e9
    },
    "U235": {
        "isotopic_frac": 0.0071,
        "heat_release_const": 5.69e-4,
        "half_life": 7.04e8
    },
    "Th232": {
        "isotopic_frac": 1.00,
        "heat_release_const": 2.64e-5,
        "half_life": 1.40e10
    },
    "K40": {
        "isotopic_frac": 1.191e-4,
        "heat_release_const": 2.92e-5,
        "half_life": 1.25e9
    }
}



In [17]:
C_Th = GRS.getConcentration(lon,lat,'th', normalized=True)
C_K = GRS.getConcentration(lon,lat,'k', normalized=True)
C_U = C_Th / 3.8
t = 4e9

H = (
    HPE['U238']['isotopic_frac'] * C_U * HPE['U238']['heat_release_const'] * math.exp( (t * math.log(2) )/(HPE['U238']['half_life']) ) +
    HPE['U235']['isotopic_frac'] * C_U * HPE['U235']['heat_release_const'] * math.exp( (t * math.log(2) )/(HPE['U235']['half_life']) ) +
    HPE['Th232']['isotopic_frac'] * C_Th * HPE['Th232']['heat_release_const'] * math.exp( (t * math.log(2) )/(HPE['Th232']['half_life']) ) +
    HPE['K40']['isotopic_frac'] * C_K * HPE['K40']['heat_release_const'] * math.exp( (t * math.log(2) )/(HPE['K40']['half_life']) )
)

H

2.825619125992331e-10

In [18]:
HPE = {
    # Constants from:
    #     - "Amagmantic Hydrothermal Systems on Mars from Radiogenic Heat", by Ojha et al. 2021,
    #     - "Geodynamics", by Turcotte & Schubert 2014.
    "U238": {
        "isotopic_frac": 0.9928, # natural abundance of this isotope relative to all isotopes of this element
        "heat_release_const": 9.46e-5, # net energy per unit mass [W/kg]
        "half_life": 4.47e9
    },
    "U235": {
        "isotopic_frac": 0.0071,
        "heat_release_const": 5.69e-4,
        "half_life": 7.04e8
    },
    "Th232": {
        "isotopic_frac": 1.00,
        "heat_release_const": 2.64e-5,
        "half_life": 1.40e10
    },
    "K40": {
        "isotopic_frac": 1.191e-4,
        "heat_release_const": 2.92e-5,
        "half_life": 1.25e9
    }
}


def calc_H(thisGRS, lon, lat, t, volatile_adjusted=True):
    """
    Calculate heat production rate in lithosphere [W/kg?] at a specific coordinate/time due to decay of radiogenic heat producing elements (U238, U235, Th232, K40).
    
    PARAMETERS:
        thisGRS : GRS type object
            Contains elemental abundances from GRS orbiter.
        lon, lat : float
            Coordinates at which to calculate H. These are used to find GRS data.
        time : float
            How many years in the past to calculate H. These are used to calculate elapsed half-lives of radiogenic HPEs.
            Ex: `t=3.4e9` would specify 3.4 billion years in the past.
        volatile_adjusted : bool
            Determine whether or not to normalize GRS radiogenic HPE concentrations to a volatile-free basis. See the next method "getAdjustedConcentration" in the GRS.py class for details.

    RETURN:
        H : float
            Heat production rate in lithosphere [W/kg?] at a specific coordinate/time due to decay of radiogenic heat producing elements (U238, U235, Th232, K40).
            
    METHODOLOGIES:
        - Hahn et al 2011: Martian surface production and crustal heat flow from Mars Odyssey Gamma-Ray spectrometry
        - Turcotte and Schubert 2001: Geodynamics (textbook)    
    """

    
    thisHPE = HPE
    
    # get concentrations of each element (these coordinates, current day)
    for element in thisHPE:
        elementname = ''.join([char for char in element if not char.isdigit()]).lower()
        if elementname == 'u':
            concentration = thisGRS.getConcentration(lon, lat, "th", volatile_adjusted) / 3.8
        else:
            concentration = thisGRS.getConcentration(lon, lat, elementname, volatile_adjusted)
    
    
        if concentration == thisGRS.getNanVal():
            return thisGRS.getNanVal()
        thisHPE[element]["concentration"] = concentration
        
    print(thisHPE)
    
    # calculate crustal heat production (these coordinates, `t` years ago)
    H = 0
    for element in thisHPE:
        H += (
            thisHPE[element]["isotopic_frac"] 
            * thisHPE[element]["concentration"] 
            * thisHPE[element]["heat_release_const"] 
            * math.exp((t * math.log(2))/(thisHPE[element]["half_life"]))
        )
    return H
            
    
calc_H(GRS, lon, lat, t)

{'U238': {'isotopic_frac': 0.9928, 'heat_release_const': 9.46e-05, 'half_life': 4470000000.0, 'concentration': 2.6614965421476245e-07}, 'U235': {'isotopic_frac': 0.0071, 'heat_release_const': 0.000569, 'half_life': 704000000.0, 'concentration': 2.6614965421476245e-07}, 'Th232': {'isotopic_frac': 1.0, 'heat_release_const': 2.64e-05, 'half_life': 14000000000.0, 'concentration': 1.0113686860160973e-06}, 'K40': {'isotopic_frac': 0.0001191, 'heat_release_const': 2.92e-05, 'half_life': 1250000000.0, 'concentration': 0.004641623858367363}}


2.825619125992331e-10